In [1]:
from collections import defaultdict
import random
import numpy as np
np.set_printoptions(precision=2, suppress=True)

import time
import copy 
import multiprocess as mp

import gym
from env import FrozenLakeCustom, FrozenLakeSimulator

from mcts_haver import run_mcts_trial
from value_iteration import value_iteration

from config import parse_args
from utils import MultiProcess

import logging
logger = logging.getLogger()
logger.setLevel(logging.FATAL)

In [2]:
np.random.seed(0)
random.seed(0)

# params
args = parse_args()

#
env_id = "FrozenLake-v1"
env = FrozenLakeCustom(
    map_name=args["map_name"], is_slippery=args["is_slippery"],
    render_mode=args["render_mode"])

simulator = FrozenLakeSimulator(env.P)

V_vit, Q_vit = value_iteration(
    simulator, args["gamma"], args["vit_thres"])
# global Q_vit_g = Q_vit
        
for state in range(simulator.num_states):
    logging.warning(f"\n-> state = {state}")
    logging.warning(f"V[state] = {V_vit[state]:0.4f}")
    for action in range(simulator.num_actions):
        logging.warning(f"Q[state][action] = {Q_vit[state][action]:0.4f}")
    logging.warning(f"best_action={np.argmax(Q_vit[state])}")
    
manager = mp.Manager()
ep_reward_list = manager.list()
Q_mcts_list = manager.list()

def run_trial(i_trial, Q_vit, args):

    random.seed(10000+i_trial)
    np.random.seed(10000+i_trial)

    env = FrozenLakeCustom(
        map_name=args["map_name"], is_slippery=args["is_slippery"],
        render_mode=args["render_mode"])

    simulator = FrozenLakeSimulator(env.P)

    Q_mcts, ep_reward = run_mcts_trial(env, simulator, Q_vit, i_trial, args)

    ep_reward_list.append(ep_reward)
    Q_mcts_list.append(Q_mcts)
    return ep_reward

In [14]:
args["update_method"] = "avg"
args["rollout_method"] = ""

print(f"num_trials = {args['num_trials']}")
# print(f"mcts_num_trajectories = {args['mcts_num_trajectories']}")

Q_mcts_dict = defaultdict()



hparam_ucb_scale_list = np.arange(10, 100, 10)
# hparam_ucb_scale_list = np.arange(20, 64, 4)
hparam_ucb_scale_list = [1, 2, 4, 8, 16, 32, 64, 128]
# hparam_ucb_scale_list = [2**i for i in range(1, 9)]


num_trajectories_list = [200, 500, 1000, 1500, 2000, 2500, 3000]
# num_trajectories_list = [200, 500, 800]
# num_trajectories_list = [2]
best_param_list = []
max_reward_mean_list = []
res_text1 = ""
res_text2 = ""
for num_trajectories in num_trajectories_list:
    print(f"\n-> num_trajectories = {num_trajectories}")
    args["mcts_num_trajectories"] = num_trajectories
    
    best_param = None
    max_reward_mean = -np.inf
    start_time = time.time()
    res_text1 += f"{num_trajectories} "
    res_text2 += f"{num_trajectories} "
    for hparam_ucb_scale in hparam_ucb_scale_list:
        # start_time = time.time()

        # print(f"hparam_ucb_scale = {hparam_ucb_scale}")
        args["hparam_ucb_scale"] = hparam_ucb_scale
        
        pool = mp.Pool()
        pool.starmap(run_trial, [(i, Q_vit, args) for i in range(args["num_trials"])])

        reward_mean = np.mean(ep_reward_list)
        reward_std = np.std(ep_reward_list, ddof=1) if len(ep_reward_list) > 1 else 0
        if hparam_ucb_scale <= 8:
            res_text1 += f"& {reward_mean:0.2f} (\u00B1{reward_std:0.2f}) "
        else:
            res_text2 += f"& {reward_mean:0.2f} (\u00B1{reward_std:0.2f}) "
        # print(f"reward = {reward_mean:0.2f} +/- {reward_std:0.2f}")

        Q_mcts_dict[f"{hparam_ucb_scale}"] = copy.deepcopy(Q_mcts_list)

        if reward_mean > max_reward_mean:
            max_reward_mean = reward_mean 
            best_param = hparam_ucb_scale
    
        ep_reward_list[:] = []
        Q_mcts_list[:] = []
    
        end_time = time.time()
        # print(f"it takes {end_time-start_time:0.4f}")
    
    res_text1 += "\\\\ \n \hline \n"
    res_text2 += "\\\\ \n \hline \n"
    
    # print(f"max_reward_mean = {max_reward_mean:0.2f}")
    # print(f"it takes {end_time-start_time:0.4f}")
    
    max_reward_mean_list.append(max_reward_mean)
    best_param_list.append(best_param)

num_trials = 20

-> num_trajectories = 200

-> num_trajectories = 500

-> num_trajectories = 1000

-> num_trajectories = 1500

-> num_trajectories = 2000

-> num_trajectories = 2500

-> num_trajectories = 3000


In [15]:
print(res_text1)
print(res_text2)

200 & -101.00 (±0.00) & -96.35 (±21.75) & -101.15 (±0.49) & -71.95 (±45.64) \\ 
 \hline 
500 & -101.00 (±0.00) & -101.10 (±0.31) & -91.40 (±29.89) & -67.05 (±47.47) \\ 
 \hline 
1000 & -101.10 (±0.45) & -101.05 (±0.22) & -91.40 (±29.89) & -47.70 (±49.46) \\ 
 \hline 
1500 & -101.00 (±0.00) & -101.05 (±0.22) & -101.25 (±0.64) & -71.90 (±45.61) \\ 
 \hline 
2000 & -101.00 (±0.00) & -101.00 (±0.00) & -96.20 (±21.70) & -72.00 (±45.45) \\ 
 \hline 
2500 & -101.05 (±0.22) & -96.15 (±21.69) & -91.30 (±29.86) & -33.30 (±45.48) \\ 
 \hline 
3000 & -101.15 (±0.49) & -101.00 (±0.00) & -86.50 (±35.56) & -47.85 (±49.33) \\ 
 \hline 

200 & -67.25 (±47.41) & -71.90 (±45.61) & -57.60 (±49.75) & -72.25 (±45.85) \\ 
 \hline 
500 & -37.95 (±47.47) & -13.70 (±29.86) & -37.95 (±47.47) & -57.35 (±49.51) \\ 
 \hline 
1000 & -18.55 (±35.54) & -4.00 (±0.00) & -4.00 (±0.00) & -23.40 (±39.81) \\ 
 \hline 
1500 & -28.25 (±43.09) & -4.00 (±0.00) & -4.00 (±0.00) & -13.70 (±29.86) \\ 
 \hline 
2000 & -23.60 (±39.71